# Data2Text

Make a start on some report generation.

In [1]:
import notebookimport

In [2]:
sr = __import__("Charts - Stage Results")
ssd = __import__("Charts - Split Sector Delta")
sp = __import__("Charts - Stage Progress")

importing Jupyter notebook from Charts - Stage Results.ipynb
importing Jupyter notebook from Charts - Split Sector Delta.ipynb
importing Jupyter notebook from Charts - Stage Progress.ipynb


In [3]:
import tracery
from tracery.modifiers import base_english
from inflectenglish import inflect_english
from pytracery_logic import pytracery_logic

def pandas_row_mapper(row, rules, root,  modifiers=base_english):
    ''' Function to parse single row of dataframe '''
    row=row.to_dict()
    rules=rules.copy()

    for k in row:
        rules[k] = str(row[k])
        
    grammar = tracery.Grammar(rules)
        
    if modifiers is not None:
        if isinstance(modifiers,list):
            for modifier in modifiers:
                grammar.add_modifiers(modifier)
        else:
            grammar.add_modifiers(modifiers)

    return grammar.flatten(root)

In [4]:
import os
import sqlite3
import pandas as pd

#dbname='wrc18.db'
#dbname='france18.db'
#conn = sqlite3.connect(dbname)

if __name__=='__main__':
    #dbname='wrc18.db'
    dbname='australia18.db'
    conn = sqlite3.connect(dbname)
    rally='Australia'
    rc='RC1'
    year=2018
    #ss='SS4'

In [5]:
import inflect
p = inflect.engine()

def timedelta_to_text(td): 
    if not td: return
    c=pd.to_timedelta(td).components
    txt=[]
    if c.hours>0:
        txt_.append(p.number_to_words(c.hours)+p.plural('hours',c.hours))
    if c.minutes>0:
        txt.append('{} {}'.format(p.number_to_words(c.minutes),p.plural('minute',c.minutes)))
    seconds=round(c.seconds+c.milliseconds/1000, 2)
    txt.append('{} {}'.format(p.number_to_words(seconds),p.plural('second',seconds)))
    return p.join(txt, final_sep="and")

def timedelta_to_numtext(td):
    if not td: return
    c=pd.to_timedelta(td).components
    txt=[]
    if c.hours>0:
        txt.append('{}{}'.format(c.hours,p.plural('hr',c.hours)))
    if c.minutes>0:
        txt.append('{}m'.format(c.minutes))
    #if c.seconds>0:
    #    seconds=c.seconds
    #    if c.milliseconds>0:
    #        seconds=round(seconds+c.milliseconds/1000, 2)
    seconds=round(c.seconds+c.milliseconds/1000, 2)
    txt.append('{}s'.format(seconds))
    return p.join(txt,conj='')

In [6]:
#https://stackoverflow.com/a/17847006/454773
from string import Formatter

def strfdelta(tdelta, fmt="{H:02}:{M:02}:{S:02.1f}"):
    f = Formatter()
    d = {}
    l = {'D': 86400, 'H': 3600, 'M': 60, 'S': 1}
    k = list(map( lambda x: x[1], list(f.parse(fmt))))
    rem = int(tdelta.total_seconds())

    for i in ('D', 'H', 'M', 'S'):
        if i in k and i in l.keys():
            d[i], rem = divmod(rem, l[i])
    d['S']=d['S']+tdelta.microseconds/1000000
    return f.format(fmt, **d)

In [7]:
import re
import isodate
import pandas as pd

def interval(timestring):
    ''' Try to return a commonly formatted time string. '''
    if timestring.startswith('PT'):
        return strfdelta(isodate.parse_duration(timestring))
    elif re.match(r"^[0-9]*:[0-9]*:[0-9\.]*$", timestring):
        return strfdelta(pd.to_timedelta(timestring))
    
    return timestring


#Then maybe register something with sqlite as part of the data ingest?
#eg https://stackoverflow.com/a/27641141/454773 ?


interval('00:04:45.5000000'), interval('PT10M40.4S'), interval('PT1H9M30.8S'), interval('PT30.8S')

('00:04:45.5', '00:10:40.4', '01:09:30.8', '00:00:30.8')

In [8]:

strfdelta(pd.to_timedelta('00:04:45.3000000'))

'00:04:45.3'

In [11]:
timedelta_to_numtext(pd.to_timedelta('00:04:45.5000000'))

'4m 45.5s'

In [12]:
stageresults = sr.dbGetRallyStages(conn, rally).sort_values('number')

#The getEnrichedStageRank query filters down to a class - default is RC1
stagerank_overall = sr.getEnrichedStageRank(conn, rally, typ='overall')

stagerank_stage = sr.getEnrichedStageRank(conn, rally, typ='stage')

In [13]:
stagerank_stage.head()

,diffFirst,diffFirstMs,diffPrev,diffPrevMs,elapsedDuration,elapsedDurationMs,entryId,position,source,stageId,...,retainedClassLead,gainedOverallPos,gainedOverallLead,overallPosDiff,lostOverallLead,retainedOverallLead,stagewin,stagewincount,winsinarow,gainedTime
0,00:00:00,0.0,00:00:00,0.0,00:04:45.5000000,285500.0,3313,1.0,Default,838,...,False,False,False,0.0,False,False,True,1.0,1,False
1,00:00:00.8000000,800.0,00:00:00.8000000,800.0,00:04:46.3000000,286300.0,3312,2.0,Default,838,...,False,False,False,0.0,False,False,False,0.0,0,False
2,00:00:01.2000000,1200.0,00:00:00.4000000,400.0,00:04:46.7000000,286700.0,3311,3.0,Default,838,...,False,False,False,0.0,False,False,False,0.0,0,False
3,00:00:01.8000000,1800.0,00:00:00.6000000,600.0,00:04:47.3000000,287300.0,3308,4.0,Default,838,...,False,False,False,0.0,False,False,False,0.0,0,False
4,00:00:02.1000000,2100.0,00:00:00.3000000,300.0,00:04:47.6000000,287600.0,3309,5.0,Default,838,...,False,False,False,0.0,False,False,False,0.0,0,False


In [14]:
stageresults.head()

,legDate,date,startListId,status,itineraryLegId,itinerarySectionId,section,order,code,distance,eventId,name,number,stageId,stageType,status,timingPrecision,itineraryLegId,itinerarySections.itinerarySectionId
0,2018-11-16,Friday 16th November,242,Completed,146,338,Section 1,1,SS1,8.77,38,Orara East I,1,838,SpecialStage,Completed,Tenth,146,338
1,2018-11-16,Friday 16th November,242,Completed,146,338,Section 1,1,SS2,14.12,38,Coldwater I,2,840,SpecialStage,Completed,Tenth,146,338
2,2018-11-16,Friday 16th November,242,Completed,146,338,Section 1,1,SS3,26.68,38,Sherwood I,3,841,SpecialStage,Completed,Tenth,146,338
3,2018-11-16,Friday 16th November,242,Completed,146,339,Section 2,2,SS4,8.77,38,Orara East II,4,844,SpecialStage,Completed,Tenth,146,339
4,2018-11-16,Friday 16th November,242,Completed,146,339,Section 2,2,SS5,14.12,38,Coldwater II,5,845,SpecialStage,Completed,Tenth,146,339


In [53]:
rules = {'onetwo_':'[branch:#_onetwo_#]#branch##condition#',
         '_onetwo_':'#stagewin._branch(True,stagefirst,stagesecond)# ',
         'y':'[switch:#sval#]#switch##condition#',
         'yy':'#drivercode# #keeploselead_# [switch:#position.int_to_words.ordinal._prefix(stage)._switch#]#switch##condition#',
         'sval':'#position.int_to_words.ordinal._prefix(stage)._switch#',
         'stagefirst':'#stagewinner_#',
         'stagethird':'took third, a further #diffPrev_txt# #backbehind# (#diffFirst_txt# off the lead) in #elapsedDuration_txt#',
         'stagefourth':'', 'stagefifth':'',
         'stagewinner_': "#won_# stage #code# #inwith_# a time of #elapsedDuration_txt#.",
         'won_':['won', 'took'],
         
         'keeploselead_':'[branch:#_keeploselead_#]#branch##condition#',
         '_keeploselead_':'#retainedOverallLead._branch(True,retlead,null)#',
         'retlead':'retaining the overall lead', 'lostlead':'losing the overall lead',
         
         'inwith_':['in', 'with'], 'backbehind':['back','behind'],
        'stagesecond':'took second, #diffFirst_txt# behind in #elapsedDuration_txt#.',
        'z':'#position._gte(1,stagewinner)#',
        'toptwo_':'[switch:#_toptwo_#]#switch##condition#','_toptwo_':'#position._gte(2,onetwo_)#',
        'null':''}

In [26]:
_code='SS5'
typ=stagerank_stage
typ['elapsedDuration_txt'] =typ['elapsedDuration'].apply(timedelta_to_numtext)
typ['diffFirst_txt'] =typ['diffFirst'].apply(timedelta_to_numtext)
typ['diffPrev_txt'] =typ['diffPrev'].apply(timedelta_to_numtext)
typ.head()

,diffFirst,diffFirstMs,diffPrev,diffPrevMs,elapsedDuration,elapsedDurationMs,entryId,position,source,stageId,...,overallPosDiff,lostOverallLead,retainedOverallLead,stagewin,stagewincount,winsinarow,gainedTime,elapsedDuration_txt,diffFirst_txt,diffPrev_txt
0,00:00:00,0.0,00:00:00,0.0,00:04:45.5000000,285500.0,3313,1.0,Default,838,...,0.0,False,False,True,1.0,1,False,4m 45.5s,0.0s,0.0s
1,00:00:00.8000000,800.0,00:00:00.8000000,800.0,00:04:46.3000000,286300.0,3312,2.0,Default,838,...,0.0,False,False,False,0.0,0,False,4m 46.3s,0.8s,0.8s
2,00:00:01.2000000,1200.0,00:00:00.4000000,400.0,00:04:46.7000000,286700.0,3311,3.0,Default,838,...,0.0,False,False,False,0.0,0,False,4m 46.7s,1.2s,0.4s
3,00:00:01.8000000,1800.0,00:00:00.6000000,600.0,00:04:47.3000000,287300.0,3308,4.0,Default,838,...,0.0,False,False,False,0.0,0,False,4m 47.3s,1.8s,0.6s
4,00:00:02.1000000,2100.0,00:00:00.3000000,300.0,00:04:47.6000000,287600.0,3309,5.0,Default,838,...,0.0,False,False,False,0.0,0,False,4m 47.6s,2.1s,0.3s


In [62]:
typ[typ['drivercode']=='LAT'][['code','position','retainedOverallLead','lostOverallLead', 'gainedOverallLead']]

,code,position,retainedOverallLead,lostOverallLead
2,SS1,3.0,False,False
12,SS2,1.0,False,False
29,SS3,6.0,False,True
40,SS4,5.0,False,False
50,SS5,3.0,False,False
62,SS6,3.0,False,False
81,SS7,10.0,False,False
91,SS8,8.0,False,False
97,SS9,2.0,False,False
109,SS10,2.0,False,False


In [27]:
typ[(typ['code']==_code)]

,diffFirst,diffFirstMs,diffPrev,diffPrevMs,elapsedDuration,elapsedDurationMs,entryId,position,source,stageId,...,overallPosDiff,lostOverallLead,retainedOverallLead,stagewin,stagewincount,winsinarow,gainedTime,elapsedDuration_txt,diffFirst_txt,diffPrev_txt
48,00:00:00,0.0,00:00:00,0.0,00:07:51.1000000,471100.0,3309,1.0,Default,845,...,-5.0,False,False,True,1.0,1,True,7m 51.1s,0.0s,0.0s
49,00:00:00.1000000,100.0,00:00:00.1000000,100.0,00:07:51.2000000,471200.0,3307,2.0,Default,845,...,-6.0,False,False,False,0.0,0,True,7m 51.2s,0.1s,0.1s
50,00:00:00.5000000,500.0,00:00:00.4000000,400.0,00:07:51.6000000,471600.0,3311,3.0,Default,845,...,-2.0,False,False,False,1.0,0,True,7m 51.6s,0.5s,0.4s
51,00:00:00.6000000,600.0,00:00:00.1000000,100.0,00:07:51.7000000,471700.0,3315,4.0,Default,845,...,-5.0,False,False,False,0.0,0,True,7m 51.7s,0.6s,0.1s
52,00:00:00.7000000,700.0,00:00:00.1000000,100.0,00:07:51.8000000,471800.0,3310,5.0,Default,845,...,-2.0,False,False,False,0.0,0,True,7m 51.8s,0.7s,0.1s
53,00:00:00.7000000,700.0,00:00:00,0.0,00:07:51.8000000,471800.0,3314,6.0,Default,845,...,3.0,False,False,False,1.0,0,True,7m 51.8s,0.7s,0.0s
54,00:00:01.6000000,1600.0,00:00:00.9000000,900.0,00:07:52.7000000,472700.0,3305,7.0,Default,845,...,3.0,False,False,False,0.0,0,False,7m 52.7s,1.6s,0.9s
55,00:00:01.8000000,1800.0,00:00:00.2000000,200.0,00:07:52.9000000,472900.0,3312,8.0,Default,845,...,7.0,True,False,False,1.0,0,False,7m 52.9s,1.8s,0.2s
56,00:00:03.6000000,3600.0,00:00:01.8000000,1800.0,00:07:54.7000000,474700.0,3306,9.0,Default,845,...,-1.0,False,False,False,0.0,0,False,7m 54.7s,3.6s,1.8s
57,00:00:22.9000000,22900.0,00:00:19.3000000,19300.0,00:08:14,494000.0,3313,10.0,Default,845,...,8.0,False,False,False,1.0,0,False,8m 14.0s,22.9s,19.3s


In [28]:
typ.columns

Index(['diffFirst', 'diffFirstMs', 'diffPrev', 'diffPrevMs', 'elapsedDuration',
       'elapsedDurationMs', 'entryId', 'position', 'source', 'stageId',
       'stageTimeId', 'status', 'class', 'code', 'distance', 'name', 'snum',
       'drivercode', 'entrantname', 'classrank', 'gainedClassPos',
       'gainedClassLead', 'classPosDiff', 'lostClassLead', 'retainedClassLead',
       'gainedOverallPos', 'gainedOverallLead', 'overallPosDiff',
       'lostOverallLead', 'retainedOverallLead', 'stagewin', 'stagewincount',
       'winsinarow', 'gainedTime', 'elapsedDuration_txt', 'diffFirst_txt',
       'diffPrev_txt'],
      dtype='object')

In [29]:
_srs=typ[(typ['code']==_code) & (typ['position']<=3)][['code','position','firstinarow','elapsedDuration',
                                                       'diffFirstMs','diffPrevMs','diffPrev_txt',
                                                    'elapsedDuration_txt', 'stagewin','diffFirst_txt',
                                                            'retainedOverallLead',       'stagewincount',
                                                                                         'drivercode']]
_srs.columns=[c.replace('.','') for c in _srs.columns]
_srs.to_dict(orient='record')

[{'code': 'SS5',
  'position': 1.0,
  'winsinarow': 1,
  'elapsedDuration': '00:07:51.1000000',
  'diffFirstMs': 0.0,
  'diffPrevMs': 0.0,
  'diffPrev_txt': '0.0s',
  'elapsedDuration_txt': '7m 51.1s',
  'stagewin': True,
  'diffFirst_txt': '0.0s',
  'retainedOverallLead': False,
  'stagewincount': 1.0,
  'drivercode': 'NEU'},
 {'code': 'SS5',
  'position': 2.0,
  'winsinarow': 0,
  'elapsedDuration': '00:07:51.2000000',
  'diffFirstMs': 100.0,
  'diffPrevMs': 100.0,
  'diffPrev_txt': '0.1s',
  'elapsedDuration_txt': '7m 51.2s',
  'stagewin': False,
  'diffFirst_txt': '0.1s',
  'retainedOverallLead': False,
  'stagewincount': 0.0,
  'drivercode': 'SUN'},
 {'code': 'SS5',
  'position': 3.0,
  'winsinarow': 0,
  'elapsedDuration': '00:07:51.6000000',
  'diffFirstMs': 500.0,
  'diffPrevMs': 400.0,
  'diffPrev_txt': '0.4s',
  'elapsedDuration_txt': '7m 51.6s',
  'stagewin': False,
  'diffFirst_txt': '0.5s',
  'retainedOverallLead': False,
  'stagewincount': 1.0,
  'drivercode': 'LAT'}]

In [30]:
_srs[_srs['position']==1].apply(lambda row: pandas_row_mapper(row, rules, "#stagewinner_#",[base_english,inflect_english,pytracery_logic]), axis=1)


48    won stage SS5 with a time of 7m 51.1s.
dtype: object

In [34]:
_srs.columns=[c.replace('.','') for c in _srs.columns]
_srs.apply(lambda row: pandas_row_mapper(row, rules, "#stagewinner#",[base_english,inflect_english,pytracery_logic]), axis=1)


48    ((stagewinner))
49    ((stagewinner))
50    ((stagewinner))
dtype: object

In [54]:
typ.columns=[c.replace('.','') for c in typ.columns]
typ[typ['position']==5 ].apply(lambda row: pandas_row_mapper(row, rules, "#yy#",[base_english,inflect_english,pytracery_logic]), axis=1)


4      NEU  
16     PAD  
28     LAP  
40     LAT  
52     PAD  
64     LAP  
76     OST  
88     LAP  
100    TÄN  
112    OST  
124    PAD  
136    OGI  
148    OST  
160    OST  
172    LAT  
184    BRE  
196    LAP  
207    SER  
220    PAD  
232    EVA  
244    OST  
257    SUN  
270    SUN  
283    LAT  
dtype: object

In [127]:
_srs.apply(lambda row: pandas_row_mapper(row, rules, "#yy#",[base_english,inflect_english,pytracery_logic]), axis=1)


56             SOR won stage SS5 in a time of 4m 14.7s.
57            LAT took second, 0.5s behind in 4m 15.2s.
58    BRE took third, a further 0.3s behind (0.8s of...
dtype: object

In [128]:
_srs[_srs['position']==1].apply(lambda row: pandas_row_mapper(row, rules, "#stagewinner_#",[base_english,inflect_english,pytracery_logic]), axis=1)


56    took stage SS5 with a time of 4m 14.7s.
dtype: object

In [111]:
ssd.dbGetSplits(conn,rally,'SS5','RC1').head()

,elapsedDuration,elapsedDurationMs,entryId,splitDateTime,splitDateTimeLocal,splitPointId,splitPointTimeId,stageTimeDuration,stageTimeDurationMs,startDateTime,startDateTimeLocal,stageId,class,code,distance,name,drivercode
0,PT1M8.494S,68494,3044,2018-10-26T13:24:08.494Z,2018-10-26T15:24:08.494+02:00,1650,49603,00:04:18.9000000,258900.0,2018-10-26T13:23:00,2018-10-26T15:23:00+02:00,835,RC1,SS5,7.0,Gandesa 2 (gravel),NEU
1,PT1M9.062S,69062,3040,2018-10-26T13:27:09.062Z,2018-10-26T15:27:09.062+02:00,1650,49604,00:04:18.6000000,258600.0,2018-10-26T13:26:00,2018-10-26T15:26:00+02:00,835,RC1,SS5,7.0,Gandesa 2 (gravel),OGI
2,PT1M8.083S,68083,3047,2018-10-26T13:30:08.083Z,2018-10-26T15:30:08.083+02:00,1650,49605,00:04:16,256000.0,2018-10-26T13:29:00,2018-10-26T15:29:00+02:00,835,RC1,SS5,7.0,Gandesa 2 (gravel),TÄN
3,PT1M8.895S,68895,3048,2018-10-26T13:33:08.895Z,2018-10-26T15:33:08.895+02:00,1650,49606,00:04:31.1000000,271100.0,2018-10-26T13:32:00,2018-10-26T15:32:00+02:00,835,RC1,SS5,7.0,Gandesa 2 (gravel),LAP
4,PT1M8.347S,68347,3046,2018-10-26T13:36:08.347Z,2018-10-26T15:36:08.347+02:00,1650,49607,00:04:15.2000000,255200.0,2018-10-26T13:35:00,2018-10-26T15:35:00+02:00,835,RC1,SS5,7.0,Gandesa 2 (gravel),LAT


In [ ]:
# if pos 1 at last split but not pos 1 at end of stage: "despite being TIME seconds up at the final split.""

### Nationalities

For example: https://blog.ouseful.info/2016/11/14/what-nationality-did-you-say-you-were-again/

In [ ]:
# ..in second, ahead of/behind fellow Swede, X <- nationality(posN)==nationality(posN-1)


#Use a similar rule for same team ahead/behind <- team(posN)==team(posN-1)

#tyreManufacturer, vehicleModel

#Maybe create enricher tables as views? More enrichers in Chart - Stage Results


In [106]:
def dbGetEntryDetail(conn, rally, rc):
    ''' Get Nationalities for drivers.               
    '''
    
    q='''
    SELECT `driver.code` drivercode, `driver.country.iso2` drivercountryISO2,
        `manufacturer.name` manufacturer, `codriver.country.iso2` codrivercountryISO2,
        tyreManufacturer, vehicleModel,`entrant.name` team,
        `codriver.abbvName` codriverabbv, `codriver.fullName` codriver,`codriver.code` codrivercode,
        `driver.abbvName` driverabbv, `driver.fullName` driver
        FROM startlists sl
    INNER JOIN championship_events ce ON sl.eventId=ce.eventId
    INNER JOIN startlist_classes sc ON sc.entryid = sl.entryId AND sc.eventId=ce.eventId
    WHERE sc.name="{rc}" AND ce.`country.name`="{rally}"
    '''.format(rc=rc, rally=rally)
    
    #q2='''
    #SELECT sp.stageId, sp.entryId, sl.`driver.code` drivercode, sp.startDateTime, i.eventId, sc.name,
    #CAST(REPLACE(code,'SS','') AS INTEGER) snum, st.position, st.diffFirstMs, st.diffPrevMs
    #FROM (SELECT DISTINCT stageId, entryId, startDateTime FROM split_times) AS sp 
    #INNER JOIN itinerary_stages i ON sp.stageId = i.stageId
    #INNER JOIN startlist_classes sc ON sc.entryid = sp.entryId AND sc.eventId=i.eventId
    #INNER JOIN championship_events ce ON i.eventId=ce.eventId
    #INNER JOIN startlists sl ON sl.entryId=sc.entryId AND sl.eventId=ce.eventId
    #INNER JOIN stage_times_stage st ON st.entryId=sc.entryId AND st.stageId = i.stageId
    #WHERE sc.name="{rc}" AND ce.`country.name`="{rally}" {stages} ORDER BY snum, sp.startDateTime
    #'''.format(rc=rc, rally=rally)
    nationalities=pd.read_sql(q,conn)
    
    #Ideally, we want this as an int, but if there are NaNs, it breaks
    #if not stagestart.empty:
    #    stagestart['startpos'] = stagestart.groupby(['snum'])['startDateTime'].cumcount()+1
    #    stagestart['classrank'] = stagestart.groupby(['snum'])['position'].rank(method='dense').astype(float)
    #else:
    #    stagestart = pd.concat([stagestart,pd.DataFrame(columns=['startpos','classrank'])],sort=False)
    #stagestart.drop(['position'], axis=1, inplace=True)
    return nationalities

In [107]:
dbGetEntryDetail(conn, rally, rc)

,drivercode,drivercountryISO2,manufacturer,codrivercountryISO2,tyreManufacturer,vehicleModel,team,codriverabbv,codriver,codrivercode,driverabbv,driver
0,OGI,FR,Ford,FR,Michelin,Fiesta WRC,M-SPORT FORD WORLD RALLY TEAM,J. INGRASSIA,Julien INGRASSIA,ING,S. OGIER,Sébastien OGIER
1,EVA,GB,Ford,GB,Michelin,Fiesta WRC,M-SPORT FORD WORLD RALLY TEAM,D. BARRITT,Daniel BARRITT,BAR,E. EVANS,Elfyn EVANS
2,SUN,FI,Ford,FI,Michelin,Fiesta WRC,M-SPORT FORD WORLD RALLY TEAM,M. MARKKULA,Mikko MARKKULA,MAR,T. SUNINEN,Teemu SUNINEN
3,MIK,NO,Hyundai,NO,Michelin,i20 Coupe WRC,HYUNDAI SHELL MOBIS WRT,A. JAEGER,Anders JAEGER,JAE,A. MIKKELSEN,Andreas MIKKELSEN
4,NEU,BE,Hyundai,BE,Michelin,i20 Coupe WRC,HYUNDAI SHELL MOBIS WRT,N. GILSOUL,Nicolas GILSOUL,GIL,T. NEUVILLE,Thierry NEUVILLE
5,PAD,NZ,Hyundai,GB,Michelin,i20 Coupe WRC,HYUNDAI SHELL MOBIS WRT,S. MARSHALL,Sebastian MARSHALL,MAR,H. PADDON,Hayden PADDON
6,LAT,FI,Toyota,FI,Michelin,Yaris WRC,TOYOTA GAZOO RACING WRT,M. ANTTILA,Miikka ANTTILA,ANT,J. LATVALA,Jari-Matti LATVALA
7,TÄN,EE,Toyota,EE,Michelin,Yaris WRC,TOYOTA GAZOO RACING WRT,M. JÄRVEOJA,Martin JÄRVEOJA,JAR,O. TÄNAK,Ott TÄNAK
8,LAP,FI,Toyota,FI,Michelin,Yaris WRC,TOYOTA GAZOO RACING WRT,J. FERM,Janne FERM,FER,E. LAPPI,Esapekka LAPPI
9,OST,NO,Citroen,NO,Michelin,C3 WRC,CITROEN TOTAL ABU DHABI WRT,T. ERIKSEN,Torstein ERIKSEN,ERI,M. OSTBERG,Mads OSTBERG
